# **딥러닝으로 AI 모델링하기**



---



## **[실습] 패션 아이템 분류기 성능 개선하기**
앞에서 tensorflow&keras로 만든 모델의 성능을 개선하기

- 이전 모델: 1개 Layer 사용
- 개선 모델: 2개 Layer 사용

In [ ]:
# 실행마다 동일한 결과를 얻기 위해 케라스에 랜덤 시드를 사용하고 텐서플로 연산을 결정적으로 만듭니다.
import tensorflow as tf

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

### **데이터 불러오기: 패션 MNIST**

In [ ]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [ ]:
# 훈련 & 검증 데이터 분리
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0              # 0~1 사이로 정규화 하기
train_scaled = train_scaled.reshape(-1, 28*28)  # 1차원 배열로 만듦

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

### **심층 신경망 만들기**

In [ ]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

model = keras.Sequential([dense1, dense2])

model.summary()

# layer1 Params : input(784) x Dense(100) + 100 = 78500
# layer2 Params : input(100) x Dense(10) + 10 = 1010
# total Params : 78500 x 1010 = 79,510

- **층을 추가하는 다른 방법**

In [ ]:
# 층 추가하는 방법
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

model.summary()

In [ ]:
# 층 추가하는 방법
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

In [ ]:
# 손실함수, 평가지표 설정
# sparse_categorical_crossentropy: 타깃값을 원-핫-인코딩으로 처리할 경우 사용(정수를 희소하게 사용)
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 학습
model.fit(train_scaled, train_target, epochs=5)

In [ ]:
# 모델 평가하기
model.evaluate(val_scaled, val_target)

### **렐루(ReLu) 활성화 함수 사용**
- 입력이 양수일 경우 활성화가 없는 것처럼 그냥 입력을 통과시키고
- 음수일 경우에는 0으로 만든다.

In [ ]:
# 첫 번째 레이어의 활성화 함수를 변경
#  sigmoid --> relu
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))   # 1차원으로 만들어준다.(학습하는 층이 아니라 층 수에 안들어감)
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0  # 0~1 사이로 정규화 하기
# flatten 층을 사용하기 때문에 데이터를 1차원으로 변경할 필요 없다.
# train_scaled = train_scaled.reshape(-1, 28*28)  # 1차원 배열로 만듦

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [ ]:
# 손실함수 적용
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 훈련
model.fit(train_scaled, train_target, epochs=5)

In [ ]:
model.evaluate(val_scaled, val_target)

# accuracy 값이 이전 모델보다 약간 상승했다.

### **옵티마이저 사용하기**

- **하이퍼파라미터**
    - 사람이 조정해줄 수 있는 파라미터
    - batch_size:
        - keras는 기본적으로 미니배치 경사 하강법을 사용하며, 미니배치 개수는 32개이다.
        - fit()메서드에서 batch_size 매개변수를 조정할 수 있다.
    - epoch

    - **옵티마이저**
        - 경사하강법 알고리즘
    - 등등

In [ ]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
sgd = keras.optimizers.SGD(learning_rate=0.1)

In [ ]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

In [ ]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

- **RMSprop : 적응적 학습률**(adaptive learning rate)
    - 모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있다. 이렇게 하면 안정적으로 최적점에 수렴할 가능성이 높다.
    - 이런 학습률을 적응적 학습률이라고 하며, 학습률 매개변수를 튜닝하는 수고를 덜 수 있다.
    - 대표적 옵티마이저: Adagrad, RMSprop

In [ ]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_scaled, train_target, epochs=5)

In [ ]:
model.evaluate(val_scaled, val_target)

# accuracy가 좀 더 개선되었습니다.



---



## **[실습] 항공사 고객 만족 분류 모델 구현 실습하기**

### 1) 데이터 로드 및 확인하기

In [ ]:
# 필요한 라이브러리 불러오기
import numpy as np
import pandas as pd
import warnings

# 경고 메시지를 무시하도록 설정하기
warnings.filterwarnings('ignore')

In [ ]:
# csv 파일에서 데이터를 로드해서 데이터프레임으로 저장하기
df = pd.read_csv('Invistico_Airline.csv')
df.head()

In [ ]:
# 데이터프레임 정보 확인하기
df.info()

In [ ]:
# 수치 데이터 , (전체 데이터: include='all')
df.describe()

In [ ]:
# 결측치 확인하기
df.isnull().sum()

### 2) 데이터 전처리하기

#### (1) 결측치 처리하기

In [ ]:
# SimpleImputer 객체로 결측치 대체하기
from sklearn.impute import SimpleImputer

mean_imputer = SimpleImputer(strategy='mean')
df["Arrival Delay in Minutes"] = mean_imputer.fit_transform(df[["Arrival Delay in Minutes"]])

#### (2) 데이터 인코딩

In [ ]:
# object 칼럼 유형을 string 유형으로 변경하기
cols = ['satisfaction', 'Gender', 'Customer Type', 'Type of Travel', 'Class']
cols = df.select_dtypes('object').columns

df[cols] = df[cols].astype(str)

In [ ]:
# 범주형 데이터를 수치값으로 변경하기
df['satisfaction'].replace(['dissatisfied','satisfied'], [0,1], inplace=True)

In [ ]:
# 순서형 인코딩(Ordinal Encoding)하기
categories = pd.Categorical(
    df['Class'],
    categories= ['Eco', 'Eco Plus', 'Business'],
    ordered=True)
labels, unique = pd.factorize(categories, sort=True)
df['Class'] = labels

In [ ]:
# 원핫 인코딩(One Hot Encoding)하기
cat_cols = ['Gender','Customer Type','Type of Travel']
df = pd.get_dummies(df, columns=cat_cols)

In [ ]:
# 데이터 전처리 결과 확인하기
df.head()

In [ ]:
# 데이터 유형 확인하기
df.dtypes

#### (3) 데이터셋 분리하기

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋을 입력(X)과 레이블(y)로 분리하기
X = df.drop(['satisfaction'], axis=1)
y = df['satisfaction'].reset_index(drop=True)

# 데이터셋을 훈련 데이터와 검증 데이터로 분리하기
X_train, X_val, y_train, y_val = train_test_split(X, y,
    test_size=0.2,
    random_state=42,
    stratify=y)

print(f'훈련 데이터셋 크기 : X_train {X_train.shape}, y_train {y_train.shape}')
print(f'검증 데이터셋 크기 : X_val {X_val.shape}, y_val {y_val.shape}')

#### (4) 데이터 스케일링하기

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 데이터 정규화하기
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

print(X_train)

### 3) 심층신경망 모델 생성하기

In [ ]:
# 필요한 라이브러리 불러오기
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import random

# 모델 시드 고정하기
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Keras의 Sequential 객체로 딥러닝 모델 구성하기
initializer = tf.keras.initializers.GlorotUniform(seed=42) #모델 시드 고정하기
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(25,),kernel_initializer=initializer))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# 모델 구조 및 파라미터 정보 확인하기
model.summary()

### 4) 모델 컴파일하기

In [ ]:
# 모델을 학습시킬 최적화 방법, loss 계산 방법, 평가 방법 설정하기
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### 5) 모델 학습하기

In [ ]:
# 모델 학습하기
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1,
                   restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=128,
          verbose=1, validation_data=(X_val, y_val), callbacks=[es])

### 6) 모델 훈련 과정 시각화하기

In [ ]:
import matplotlib.pyplot as plt

# 훈련 과정 정확도(accuracy) 시각화하기
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

# 훈련 과정 손실(loss) 시각화하기
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()